In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [2]:
def get_reviews(appid, params):
        url_start = 'https://store.steampowered.com/appreviews/'
        response = requests.get(url=url_start+appid, params=params, headers={'User-Agent': 'Mozilla/5.0'})
        return response.json() # return data extracted from the json response

In [3]:
def get_n_reviews(appid, n=100):
    reviews = []
    cursor = '*'
    params = { # https://partner.steamgames.com/doc/store/getreviews
            'json' : 1,
            'filter' : 'all', # sort by: recent, updated, all (helpfullness)
            'language' : 'english', # https://partner.steamgames.com/doc/store/localization
            'day_range' : 9223372036854775807, # shows reveiws from all time
            'review_type' : 'all', # all, positive, negative
            'purchase_type' : 'all', # all, non_steam_purchase, steam
        }
    while n > 0:
        params['cursor'] = cursor.encode() # for pagination
        params['num_per_page'] = min(100, n) # 100 is the max possible reviews in one requests
        n -= 100
        
        response = get_reviews(appid, params)
        cursor = response['cursor']
        reviews += response['reviews']
        
        if len(response['reviews']) < 100: break
    
    return reviews

In [4]:
def get_n_appids(n=100, filter_by='topsellers'):
    appids = []
    url = f'https://store.steampowered.com/search/?category1=998&filter={filter_by}&page='
    page = 0
    
    while page*25 < n:
        page += 1
        response = requests.get(url=url+str(page), headers={'User-Agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(response.text, 'html.parser')
        for row in soup.find_all(class_='search_result_row'):
            appids.append(row['data-ds-appid'])
    
    return appids[:n]

In [5]:
reviews = []
appids = get_n_appids(750)
for appid in appids:
    reviews += get_n_reviews(appid, 100)
df = pd.DataFrame(reviews)[['review', 'voted_up']]
df

,review,voted_up
0,I wanted to wait until I had 100 hours into th...,True
1,"I don't know how these devs did it, but I have...",True
2,"Has more game play, less bugs, and is polished...",True
3,I am very impressed with this game. Its worth...,True
4,Imagine if Rust and Runescape had a baby (with...,True
...,...,...
73091,"70 hours in. No crashes, no slow-mo glitches, ...",True
73092,We still need a good GM mode.,False
73093,WWE 2K19 is a wrestling simulation game. It's ...,True
73094,"Alrightie, where do I begin?\nI fell in love w...",False


In [6]:
df.dropna(inplace=True)
df.reset_index(inplace=True)
df.voted_up.value_counts(normalize=True)

True     0.805858
False    0.194142
Name: voted_up, dtype: float64

In [7]:
df.to_feather('../data/reviews_raw.feather')